# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: fe96489a-3af3-4479-8e0e-4cffc39f32ea
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session fe96489a-3af3-4479-8e0e-4cffc39f32ea to get into ready status...
Session fe96489a-3af3-4479-8e0e-4cffc39f32ea ha

#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [ ]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='etl-project-for-database', table_name='raw_data')
dyf.printSchema()

#### Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [6]:
df = dyf.toDF()
df.show(10)

+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+
|  id|year_birth| education|marital_status|income|kidhome|teenhome|dt_customer|recency|mntwines|mntfruits|mntmeatproducts|mntfishproducts|mntsweetproducts|mntgoldprods|numdealspurchases|numwebpurchases|numcatalogpurchases|numstorepurchases|numwebvisitsmonth|acceptedcmp3|acceptedcmp4|acceptedcmp5|acceptedcmp1|acceptedcmp2|complain|z_costcontact|z_revenue|response|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+----------

#### Drop columns that we don't need it


In [7]:
df = df["id","year_birth","education","marital_status","income","dt_customer"]
df.show(10)

+----+----------+----------+--------------+------+-----------+
|  id|year_birth| education|marital_status|income|dt_customer|
+----+----------+----------+--------------+------+-----------+
|5524|      1957|Graduation|        Single| 58138| 2012-09-04|
|2174|      1954|Graduation|        Single| 46344| 2014-03-08|
|4141|      1965|Graduation|      Together| 71613| 2013-08-21|
|6182|      1984|Graduation|      Together| 26646| 2014-02-10|
|5324|      1981|       PhD|       Married| 58293| 2014-01-19|
|7446|      1967|    Master|      Together| 62513| 2013-09-09|
| 965|      1971|Graduation|      Divorced| 55635| 2012-11-13|
|6177|      1985|       PhD|       Married| 33454| 2013-05-08|
|4855|      1974|       PhD|      Together| 30351| 2013-06-06|
|5899|      1950|       PhD|      Together|  5648| 2014-03-13|
+----+----------+----------+--------------+------+-----------+
only showing top 10 rows


#### Check NaN values for each column


In [8]:
from pyspark.sql.functions import *

df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+---+----------+---------+--------------+------+-----------+
| id|year_birth|education|marital_status|income|dt_customer|
+---+----------+---------+--------------+------+-----------+
|  0|         0|        0|             0|    24|          0|
+---+----------+---------+--------------+------+-----------+


There are 24 NaN values in "income" column. Let's fill NaN values with mean.

In [9]:
# Calculate the mean value of the column
mean_value = df.select(mean(col('income'))).collect()[0][0]

# Fill missing values with the mean value
df = df.fillna(mean_value, subset=['income'])

# Check
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+---+----------+---------+--------------+------+-----------+
| id|year_birth|education|marital_status|income|dt_customer|
+---+----------+---------+--------------+------+-----------+
|  0|         0|        0|             0|     0|          0|
+---+----------+---------+--------------+------+-----------+


#### Write the data to our S3 Bucket named "transformed_data" as csv.

In [10]:
df.write \
    .format("csv") \
    .mode("append") \
    .option("header", "true") \
    .save("s3://etl-bucket-s3/etl-bucket-s3-database/transformed_data/")

#### Write the data to our S3 Bucket named "transformed_data" as json.

In [ ]:
df.write \
    .format("json") \
    .mode("append") \
    .save("s3://etl-project-for-medium/etl-project-for-medium-database/transformed_data/")